In [164]:
import pandas as pd
import sys

In [166]:
# news = pd.read_csv('results/news.csv')
news_tweets = pd.read_csv('results/older_sentiment/news_tweets.csv')
# user_resp = pd.read_csv('results/user_response.csv')

In [3]:
i = 1

In [167]:
news_tweets.shape

(34790, 36)

In [12]:
news[['title', 'trump_neg', 'trump_neu', 'trump_pos', 'biden_neg', 'biden_neu', 'biden_pos']].head(5)

,title,trump_neg,trump_neu,trump_pos,biden_neg,biden_neu,biden_pos
0,Trump Reacts to Romney Being Isolated for Poss...,0.00000,0.943155,0.000000,0.0,0.000000,0.0
1,Trump Signs Executive Order to Prevent Hoardin...,0.00000,0.000000,0.535006,0.0,0.000000,0.0
2,Donald Trump: 'We Cannot Let the Cure Be Worse...,0.00000,0.737229,0.000000,0.0,0.000000,0.0
3,Man with Coronavirus Credits Drug Touted by Tr...,0.00000,0.000000,0.612008,0.0,0.000000,0.0
4,Biden: Trump and McConnell Trying to Put a Cor...,0.54157,0.000000,0.000000,0.0,0.572135,0.0


In [36]:
news.title[0][-6:-1]

'aviru'

In [20]:
news.title.str.contains('Trump').sum()

27650

In [169]:
(news_tweets_filtered.text.str.contains('Trump') | news_tweets_filtered.text.str.contains('Biden')).value_counts()

True    26412
Name: text, dtype: int64

In [168]:
news_tweets_filtered = news_tweets[news_tweets.text.str.contains('Trump') | news_tweets.text.str.contains('Biden')]

In [174]:
ent = 'Biden' #'Biden']
proc_head = []
senti = []
j = 0
for i, n in enumerate(news_tweets_filtered.text):
    # print(type(n))
    k = n.split(ent)
    # print(k)
    sn = dict()
    j+=1
    if(ent in n):
        # print(x)
        # proc_head.append(x)
        x = [k[0], ent, k[1]]
        sentiment = tsc.infer_from_text(k[0].strip('\n'), ent, k[1])
        # print(x, sentiment)
        # print(sentiment[0]['class_label'], sentiment[0]['class_prob'])
        sn[sentiment[0]['class_label']] = sentiment[0]['class_prob']
        sn[sentiment[1]['class_label']] = sentiment[1]['class_prob']
        sn[sentiment[2]['class_label']] = sentiment[2]['class_prob']
        senti.append(sn)
        print(j, end = "\r")
        # sys.exit()
    else:
        senti.append(None)

In [177]:
len(senti), news_tweets_filtered.shape

(26412, (26412, 42))

In [178]:
pos_t_senti = [x['positive'] if x != None else None for x in senti]
neg_t_senti = [x['negative'] if x != None else None for x in senti]
neu_t_senti = [x['neutral'] if x != None else None for x in senti]

In [179]:
news_tweets_filtered['biden_pos_new'] = pos_t_senti
news_tweets_filtered['biden_neg_new'] = neg_t_senti
news_tweets_filtered['biden_neu_new'] = neu_t_senti

/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [180]:
news_tweets_filtered.to_csv('results/news_tweets.csv')

In [162]:
news_tweets_filtered.columns, news_tweets_filtered.shape

(Index(['Unnamed: 0', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
        'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'tweet_id', 'conversation_id',
        'author_id', 'created_at', 'geo', 'lang', 'like_count', 'quote_count',
        'reply_count', 'retweet_count', 'source', 'text', 'display_name',
        'topics', 'topic_labels', 'theme', 'pos_senti', 'neu_senti',
        'neg_senti', 'publication', 'date', 'week', 'polIncl', 'hashtag',
        'clean_text', 'proc_text', 'Segmented#', 'processed_text', 'topic_ids',
        'subtopic', 'biden_pos_new', 'biden_neg_new', 'biden_neu_new',
        'trump_pos_new', 'trump_neg_new', 'trump_neu_new'],
       dtype='object'),
 (26412, 42))

In [89]:
# len(pos_t_senti)

In [51]:
# from NewsSentiment import TargetSentimentClassifier
# for x in proc_head:
#     # print(type(x))
#     tsc = TargetSentimentClassifier()
#     sentiment = tsc.infer_from_text("" ,"Mark Meadows", "'s coverup of Trump’s coup attempt is falling apart.")
#     print(sentiment[0])